In [2]:
import itertools as it


In [3]:
# {
# #     "got_types_from_context" : false,
# #     "retag_treatment" : null,
# #     "age_group_from_session" : {},  
# #     "user_gender" : null,    
#     "age_group" : null,
# #     "other_tags" : {},    
#     "sentiment" : "neutral",
    
#     "is_a_question" : false,
#     "is_yes_no_question" : null,
#     "is_a_punctuation_mark" : false,

# #    "vertical" : "fashion",      
# #    "subvertical_from_user_session" : "lingerie",
#     "subvertical_tag" : "lingerie",
# #    "user_current_subvertical" : "lingerie",
    
#     "type_parents" : ["bra"],
#     "product_types" : [ "bra" ]
#     "original_product_types" : ["bra"],
#     "product_type_tags" : [ "bra"],
#     "type_synonyms" : { "bra" : ["everyday bra", "bra"] },
#       "negative_product_type_tags" : [],
#       "positive_product_type_tags" : ["bra"],
#     "product_types_from_user_session" : ["bra"],
#     "product_type_sentiment" : "neutral",
#     "product_desc_tags" : [],  
#     "product_ids" : null,
    
#     "brands" : [],
#     "brand_tags" : [],
#     "original_brand_tags" : [],
#     "positive_brand_tags" : [],
#     "negative_brand_tags" : [],  
        
        
#      "negative_aspects" : [],
#      "positive_aspects" : [],

#     "purpose_tags" : [ "unambiguous" ],
# #     "purpose_tags_general_data" : {"vertical_purposes" : [],"purpose_tag_unambiguous" : "unambiguous"},
    

#     "occasion_tags" : { "occasions" : [],"dress_codes" : [],},
#     "original_occasion_words" : [],
# } 

In [16]:
def get_vals_unique_sorted(dic, key, default=[], extractor=None, fixer=None):    
    if extractor is None:
        extractor = lambda v: v   
    if fixer is None:
        fixer = lambda v:v
    try:
        val = dic.get(key,default)
        val = [val] if isinstance(val, str) else val
        vals = map(extractor,val)
        unique = set(vals)
        unique_sorted = fixer(sorted(unique)) 
        return unique_sorted
#         return list(unique_sorted)
    except Exception as e:
        print(e)
#         import pdb; pdb.set_trace()


In [17]:
def get_fixed_length(N, arr, default='*'):   
    #import pdb; pdb.set_trace()
    if len(arr) >= N:
        return arr[:N]
    else:
        return arr + [default] * (N - len(arr))  

In [11]:
import functools as ft
import numpy as np

class Message:   
    def __init__(self, msg_hash):
        self._msg = msg_hash
                    
    def extract_aspect_value(self, aspect):
        try:
            if isinstance(aspect, str):
                return aspect
            else:
                return aspect['aspect']
        except Exception as e:
            print(aspect)            
        
    def get_fields_for_vector(self, fixed_length=None, fixed_length_by_field={}, fld_name_suffix=None):
        
        msg = self._msg['msg']       
        
        msg_array_geter = ft.partial(get_vals_unique_sorted, msg)
        def msg_array_geter_by_field(field_name, extractor=None):            
            fixed_length_for_field = fixed_length_by_field.get(field_name,fixed_length)
            fixer = ft.partial(get_fixed_length, fixed_length_for_field) if fixed_length_for_field else None            
            return msg_array_geter(field_name, fixer=fixer, extractor=extractor)
        
        
         
        f = {}
        f['text'] = msg_array_geter_by_field('text') 

#         if fixed_length:            
#             msg_array_geter =  ft.partial(msg_array_geter, fixer=ft.partial(get_fixed_length,fixed_length))
            
    #     f['product_types'] = msg['product_types']
    #     f['type_parents'] = msg['type_parents']
    
      
    
        f['positive_product_type_tags'] = msg_array_geter_by_field('positive_product_type_tags') 
        f['negative_product_type_tags'] = msg_array_geter_by_field('negative_product_type_tags')

    #     brand_tags
    #     positive_brand_tags
    #     original_brand_tags
    #     f['sentiment'] = msg['sentiment']
    #     relative_filters
   
        f['positive_aspects'] = msg_array_geter_by_field('positive_aspects', extractor=self.extract_aspect_value)
        f['negative_aspects'] = msg_array_geter_by_field('negative_aspects', extractor=self.extract_aspect_value)
    #     f['positive_aspects'] = 'zzz'#msg['positive_aspects']
    
        if fld_name_suffix:
            f = {f"{key}{fld_name_suffix}": f.pop(key) for key in list(f)}

        
        return list(f.items())


In [12]:
def create_embeddings_cache_query(text, metadata):
    _metadata = metadata.copy()
    q = {
        **{'text':text},        
        **{f"metadata.{key}": _metadata.pop(key) for key in list(_metadata)}
    }
    return q